<a href="https://colab.research.google.com/github/slotport/public-colabnotebooks/blob/main/snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# @Google Driveをマウントする

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# @sshサーバを起動する

In [ ]:
!apt -y install openssh-server
!service ssh start

In [ ]:
import os
import re
home = os.environ["HOME"]
try:
  os.makedirs(f"{home}/.ssh")
except FileExistsError:
  pass
os.chmod(f"{home}/.ssh", 0o700)
key = ""
exist = False
with open("/content/drive/MyDrive/colab_server/authorized_key_path.txt") as pathfile:
  with open(pathfile.read().strip()) as keyfile:
    _ = keyfile.read()
    _ = re.match(r"ssh-\S+\s+\S+", _)
    if _:
      key = _[0]
if key:
  print(key)
  try:
    with open(f"{home}/.ssh/authorized_keys") as authkeysfile:
      for _ in authkeysfile.readlines():
        if key in _:
          exist = True
  except FileNotFoundError:
    pass
if not exist:
  with open(f"{home}/.ssh/authorized_keys", mode="at") as authkeysfile:
    authkeysfile.write(key + "\n")
else:
  print("already exists")
os.chmod(f"{home}/.ssh/authorized_keys", 0o600)

# @ngrokに公開する

In [ ]:
!test -f ngrok || curl -sL https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tar.gz | tar xzf - ngrok > ngrok
!chmod 755 ngrok
!ls -l ngrok

In [ ]:
!./ngrok authtoken $(cat /content/drive/MyDrive/colab_server/ngrok_authtoken.txt)

In [ ]:
import os
get_ipython().system_raw('bash -c "./ngrok tcp 22"   &')


In [ ]:
import requests
import urllib.parse

def get_ngrok_info():
  return requests.get('http://localhost:4040/api/tunnels').json()

def get_ngrok_tunnels():
  for tunnel in get_ngrok_info()['tunnels']:
    name = tunnel['name']
    yield name, tunnel

def get_ngrok_tunnel(name):
  for name1, tunnel in get_ngrok_tunnels():
    if name == name1:
      return tunnel

def get_ngrok_url(name, local=False):
  if local:
    return get_ngrok_tunnel(name)['config']['addr']
  else:
    return get_ngrok_tunnel(name)['public_url']

In [ ]:
get_ngrok_url("command_line")